In [4]:

import pandas as pd
import pickle
from scipy.stats import poisson


In [5]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [7]:
dict_table['Group C']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Argentina,0,0,0,0,0,0,0,0
1,2,Saudi Arabia,0,0,0,0,0,0,0,0
2,3,Mexico,0,0,0,0,0,0,0,0
3,4,Poland,0,0,0,0,0,0,0,0


In [8]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


In [9]:
df_fixture 

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [11]:
#spilt of into df_home and df_away


df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

#df_home = df_historial_data[['HomeTeam','HomeGoals','AwayGoals']]

In [12]:
df_home

,HomeTeam,HomeGoals,AwayGoals
0,France,4,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
895,Brazil,2,0
896,Serbia,1,2
897,Serbia,0,2
898,France,1,0


In [15]:
# updating col names for better understanding

df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScore','AwayGoals':'GoalsConceded'})
df_away = df_home.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScore'})

In [16]:
df_away

,Team,GoalsScore,GoalsConceded
0,France,4,1
1,Uruguay,4,2
2,Uruguay,6,1
3,Argentina,6,1
4,Paraguay,1,0
...,...,...,...
895,Brazil,2,0
896,Serbia,1,2
897,Serbia,0,2
898,France,1,0


In [19]:
# now for calculating the team stregnth by finding mean of goal score & concede

df_team_strength_checker = pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()

In [20]:
df_team_strength_checker

,GoalsScore,GoalsConceded
Team,,
Algeria,0.833333,1.666667
Angola,0.000000,1.000000
Argentina,1.913793,0.862069
Australia,1.000000,1.857143
Austria,2.285714,1.571429
...,...,...
Uruguay,2.060606,1.121212
Wales,2.000000,1.000000
West Germany,2.500000,0.894737


In [21]:
# create a function that determines how many points a team will get

def predict_points(home, away):
    if home in df_team_strength_checker.index and away in df_team_strength_checker.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength_checker.at[home,'GoalsScore'] * df_team_strength_checker.at[away,'GoalsConceded']
        lamb_away = df_team_strength_checker.at[away,'GoalsScore'] * df_team_strength_checker.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team here'x' num of goals
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)#three conditions check below
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [29]:
# testing the funtion
# predict_points('Argentina','Poland')
# predict_points('Brazil','Costa Rica')

# qatar platying first time.no historic data
predict_points('Qatar (H)','Ecuador')

(0, 0)

In [30]:
#dividing the fixtures into group stage,round of 16,quater_final,semi,final

df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()



In [31]:
df_fixture_group_48

,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022


In [32]:
dict_table['Group C']


,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Argentina,0,0,0,0,0,0,0,0
1,2,Saudi Arabia,0,0,0,0,0,0,0,0
2,3,Mexico,0,0,0,0,0,0,0,0
3,4,Poland,0,0,0,0,0,0,0,0


In [33]:
# updating the group stage points
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [40]:
dict_table['Group G']

,Team,Pts
0,Brazil,8.0
1,Switzerland,6.0
2,Cameroon,3.0
3,Serbia,1.0


In [41]:
dict_table['Group H']

,Team,Pts
0,Portugal,7.0
1,Uruguay,6.0
2,South Korea,2.0
3,Ghana,2.0


In [ ]:
# looking forward to knockout stages


In [42]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [43]:
# assigning the points and fixture the team for round of 16
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'


In [44]:
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Mexico,2022,?
51,England,Match 51,Ecuador,2022,?
52,Germany,Match 53,Croatia,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Belgium,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [45]:
#quater final winner can also predict using the "predict_points" function

def get_winner(df_fixture_updated):
    for index,row in df_fixture_updated.iterrows():
        home,away = row['home'],row['away']
        points_home,points_away = predict_points(home,away)
        if points_home>points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index,'winner']=winner
    return df_fixture_updated
            


In [46]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Mexico,2022,France
51,England,Match 51,Ecuador,2022,England
52,Germany,Match 53,Croatia,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Belgium,Match 55,Spain,2022,Belgium
55,Portugal,Match 56,Switzerland,2022,Portugal


In [47]:
# quater_finals predict
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [48]:
# upating result.df_fixture_round_1=its like quartar result..
# df_fixture_round_2= its like semi result
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2


In [49]:
update_table(df_fixture_knockout, df_fixture_quarter)


,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Belgium,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [50]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Argentina
58,Belgium,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,England


In [51]:
# semi predict

update_table(df_fixture_quarter, df_fixture_semi)

 

,home,score,away,year,winner
60,Argentina,Match 61,Brazil,2022,?
61,England,Match 62,Portugal,2022,?


In [52]:

get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Argentina,Match 61,Brazil,2022,Brazil
61,England,Match 62,Portugal,2022,England


In [53]:
# final
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,England,2022,?


In [54]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,England,2022,England
